In [1]:
# !pip install tqdm

In [2]:
# !pip install pycaret

In [3]:
# !pip install pycaret[all]

In [4]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm
tqdm.pandas()

import os
from pprint import pprint
import re
from collections import defaultdict

In [5]:
from datetime import datetime

print("Begining Time:\n", datetime.now())

Begining Time:
 2024-02-11 12:18:51.266952


In [6]:
DATA_ADDRESS = "./data"
MODEL_DIR = "./model/test_source"
os.listdir(DATA_ADDRESS)

['.gitkeep',
 'correlation_df_mean_no_clean.csv',
 'correlation_df_std_no_clean.csv',
 'df_5s.pkl',
 'df_5s_source.pkl',
 'df_index_test.pkl',
 'df_index_train.pkl',
 'df_merged.pkl',
 'df_merged_pivot.xlsx',
 'df_processed_descriptive_statistics.pickle',
 'df_processed_descriptive_statistics.pkl',
 'df_processed_descriptive_statistics_mel40_mfcc20.pkl',
 'df_processed_simple.pkl',
 'df_processed_stat_no_trim_cleaned_fft2048_mel128_mfcc20.pkl',
 'df_processed_stat_no_trim_cleaned_fft2048_mel40_mfcc17.pkl',
 'df_processed_stat_no_trim_cleaned_fft512_mel64_mfcc17.pkl',
 'df_processed_stat_no_trim_no_clean_fft2048_mel128_mfcc20.pkl',
 'df_processed_stat_no_trim_no_clean_fft512_mel128_mfcc17.pkl',
 'preprocessed',
 'spearman_correlation_df_mean_cleaned.csv',
 'spearman_correlation_df_mean_no_clean.csv',
 'spearman_correlation_df_std_cleaned.csv',
 'spearman_correlation_df_std_no_clean.csv',
 'VOiCES_devkit']

In [7]:
TRAIN_DIR = os.path.join(DATA_ADDRESS,'preprocessed','train')
os.listdir(TRAIN_DIR)

['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12500-12799.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6499.pkl',
 'mel_len15_fft2048_mels1

## Load data

In [8]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj
def load_train(train_files):
    ls_X_trian = []
    for k,v in tqdm(train_files.items()):  
        train_file = v['file_name']
        file_dir = os.path.join(TRAIN_DIR, train_file)
        ls_X_trian.append(load_pickle(file_dir))
    return np.concatenate(ls_X_trian, axis=0)

### X_train

In [9]:

file_prefix = 'mfcc_len5_fft2048_mels128_mfcc17_'

# List all files in the directory
files = os.listdir(TRAIN_DIR)
pprint(files)



['mel_len15_fft2048_mels128_mfcc17_0-499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_12500-12799.pkl',
 'mel_len15_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2000-2499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_2500-2999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3000-3499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_3500-3999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4000-4499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_4500-4999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_500-999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5000-5499.pkl',
 'mel_len15_fft2048_mels128_mfcc17_5500-5999.pkl',
 'mel_len15_fft2048_mels128_mfcc17_6000-6499.pkl',
 'mel_len15_fft2048_mels1

In [10]:
# Filter files that start with the specified prefix
train_files_mfcc = [file for file in files if file.startswith(file_prefix)]
pprint(train_files_mfcc)

['mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_1000-1499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_10000-10499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_10500-10999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_11000-11499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_11500-11999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_12000-12499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_12500-12999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_13000-13499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_13500-13999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_14000-14499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_14500-14999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_1500-1999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_15000-15499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_15500-15999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_16000-16499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_16500-16999.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_17000-17499.pkl',
 'mfcc_len5_fft2048_mels128_mfcc17_17500-17999.pkl',
 'm

In [11]:
# Regular expression to match two numbers at the end of the file name
pattern = re.compile(r'_(\d+)-(\d+)\.pkl$')
n_interval = 500

# Extract the numbers
dir_files = defaultdict(dict)
for file_name in train_files_mfcc:
    match = pattern.search(file_name)
    n1 = int(match.group(1))
    n2 = int(match.group(2))
    file_num = n1/n_interval
    dir_files[file_num] = {
        'begin':n1,
        'end':n2,
        'file_name' : file_name,
    }
sorted_dir_files = {k: dir_files[k] for k in sorted(dir_files)}
pprint(sorted_dir_files)

{0.0: {'begin': 0,
       'end': 499,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_0-499.pkl'},
 1.0: {'begin': 500,
       'end': 999,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_500-999.pkl'},
 2.0: {'begin': 1000,
       'end': 1499,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_1000-1499.pkl'},
 3.0: {'begin': 1500,
       'end': 1999,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_1500-1999.pkl'},
 4.0: {'begin': 2000,
       'end': 2499,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_2000-2499.pkl'},
 5.0: {'begin': 2500,
       'end': 2999,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_2500-2999.pkl'},
 6.0: {'begin': 3000,
       'end': 3499,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_3000-3499.pkl'},
 7.0: {'begin': 3500,
       'end': 3999,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_3500-3999.pkl'},
 8.0: {'begin': 4000,
       'end': 4499,
       'file_name': 'mfcc_len5_fft2048_mels128_mfcc17_4000-4499.pk

In [12]:
X = load_train(sorted_dir_files)
X.shape

 41%|████      | 41/100 [00:00<00:00, 410.00it/s]

100%|██████████| 100/100 [00:00<00:00, 382.19it/s]


(49664, 17, 216)

### y_train

In [13]:
df_raw = pd.read_pickle(os.path.join(DATA_ADDRESS,'df_5s.pkl'))
df_raw

,origin_folder,speaker,distractor,room,category,filename
0,distant-16k/speech/test/rm3/musi/sp0192,0192,musi,rm3,test,5seconds-16k/speech/test/rm3/musi/sp0192/Lab41...
1,distant-16k/speech/test/rm3/musi/sp0192,0192,musi,rm3,test,5seconds-16k/speech/test/rm3/musi/sp0192/Lab41...
2,distant-16k/speech/test/rm3/musi/sp0192,0192,musi,rm3,test,5seconds-16k/speech/test/rm3/musi/sp0192/Lab41...
3,distant-16k/speech/test/rm3/musi/sp0192,0192,musi,rm3,test,5seconds-16k/speech/test/rm3/musi/sp0192/Lab41...
4,distant-16k/speech/test/rm3/musi/sp0192,0192,musi,rm3,test,5seconds-16k/speech/test/rm3/musi/sp0192/Lab41...
...,...,...,...,...,...,...
74459,distant-16k/speech/test/rm2/babb/sp2137,2137,babb,rm2,test,5seconds-16k/speech/test/rm2/babb/sp2137/Lab41...
74460,distant-16k/speech/test/rm2/babb/sp2137,2137,babb,rm2,test,5seconds-16k/speech/test/rm2/babb/sp2137/Lab41...
74461,distant-16k/speech/test/rm2/babb/sp2137,2137,babb,rm2,test,5seconds-16k/speech/test/rm2/babb/sp2137/Lab41...
74462,distant-16k/speech/test/rm2/babb/sp2137,2137,babb,rm2,test,5seconds-16k/speech/test/rm2/babb/sp2137/Lab41...


In [14]:
df_raw[(df_raw['category'] == 'train')]['speaker'].unique()

array(['1121', '6848', '7095', '5789', '1050', '6788', '0032', '4331',
       '8152', '3235', '4116', '3994', '0208', '0492', '5338', '8222',
       '1851', '7445', '7540', '2289', '2060', '0240', '1867', '0083',
       '5740', '4848', '1259', '3521', '2764', '1116', '0224', '2074',
       '6319', '0403', '8108', '1963', '3368', '0480', '0250', '5802',
       '1182', '0652', '5157', '0296', '7276', '0472', '4160', '0868',
       '0887', '5678', '0196', '6395', '4441', '5583', '0254', '0188',
       '7704', '4057', '2803', '4859', '1066', '8425', '7867', '6147',
       '0637', '0198', '2269', '4590', '8266', '0949', '2149', '7264',
       '5935', '7868', '2162', '3645', '0204', '7000', '5968', '7688',
       '2110', '5635', '1961', '0209', '1737', '1160', '4586', '1607',
       '4967', '8057', '0248', '8635', '0205', '0948', '0770', '6099',
       '1383', '7517', '4535', '0288', '5400', '1472', '1392', '4839',
       '5456', '1052', '8051', '0174', '3835', '0307', '4110', '6696',
      

In [15]:
df_raw[(df_raw['category'] == 'train')]['speaker'].unique()[9]

'3235'

In [16]:
df_raw[(df_raw['category'] == 'train')& (df_raw['speaker'] == '3235')]

,origin_folder,speaker,distractor,room,category,filename
280,distant-16k/speech/train/rm3/musi/sp3235,3235,musi,rm3,train,5seconds-16k/speech/train/rm3/musi/sp3235/Lab4...
281,distant-16k/speech/train/rm3/musi/sp3235,3235,musi,rm3,train,5seconds-16k/speech/train/rm3/musi/sp3235/Lab4...
282,distant-16k/speech/train/rm3/musi/sp3235,3235,musi,rm3,train,5seconds-16k/speech/train/rm3/musi/sp3235/Lab4...
283,distant-16k/speech/train/rm3/musi/sp3235,3235,musi,rm3,train,5seconds-16k/speech/train/rm3/musi/sp3235/Lab4...
284,distant-16k/speech/train/rm3/musi/sp3235,3235,musi,rm3,train,5seconds-16k/speech/train/rm3/musi/sp3235/Lab4...
...,...,...,...,...,...,...
72831,distant-16k/speech/train/rm4/none/sp3235,3235,none,rm4,train,5seconds-16k/speech/train/rm4/none/sp3235/Lab4...
72832,distant-16k/speech/train/rm4/none/sp3235,3235,none,rm4,train,5seconds-16k/speech/train/rm4/none/sp3235/Lab4...
72833,distant-16k/speech/train/rm4/none/sp3235,3235,none,rm4,train,5seconds-16k/speech/train/rm4/none/sp3235/Lab4...
72834,distant-16k/speech/train/rm4/none/sp3235,3235,none,rm4,train,5seconds-16k/speech/train/rm4/none/sp3235/Lab4...


In [17]:
df_raw[df_raw['category'] == 'train']

,origin_folder,speaker,distractor,room,category,filename
30,distant-16k/speech/train/rm4/musi/sp1121,1121,musi,rm4,train,5seconds-16k/speech/train/rm4/musi/sp1121/Lab4...
31,distant-16k/speech/train/rm4/musi/sp1121,1121,musi,rm4,train,5seconds-16k/speech/train/rm4/musi/sp1121/Lab4...
32,distant-16k/speech/train/rm4/musi/sp1121,1121,musi,rm4,train,5seconds-16k/speech/train/rm4/musi/sp1121/Lab4...
33,distant-16k/speech/train/rm4/musi/sp1121,1121,musi,rm4,train,5seconds-16k/speech/train/rm4/musi/sp1121/Lab4...
34,distant-16k/speech/train/rm4/musi/sp1121,1121,musi,rm4,train,5seconds-16k/speech/train/rm4/musi/sp1121/Lab4...
...,...,...,...,...,...,...
74443,distant-16k/speech/train/rm3/tele/sp2764,2764,tele,rm3,train,5seconds-16k/speech/train/rm3/tele/sp2764/Lab4...
74444,distant-16k/speech/train/rm3/tele/sp2764,2764,tele,rm3,train,5seconds-16k/speech/train/rm3/tele/sp2764/Lab4...
74445,distant-16k/speech/train/rm3/tele/sp2764,2764,tele,rm3,train,5seconds-16k/speech/train/rm3/tele/sp2764/Lab4...
74446,distant-16k/speech/train/rm3/tele/sp2764,2764,tele,rm3,train,5seconds-16k/speech/train/rm3/tele/sp2764/Lab4...


In [18]:
y = np.array(df_raw[df_raw['category']=='train']['speaker']).astype('float32')
print(y.shape)
y

(49664,)


array([1121., 1121., 1121., ..., 2764., 2764., 2764.], dtype=float32)

In [19]:
# X = X[:1000]
# y = y[:1000]

In [20]:
unique, counts = np.unique(y, return_counts=True)
print(len(unique))
dict(zip(unique, counts))


200


{32.0: 256,
 83.0: 256,
 93.0: 256,
 112.0: 256,
 122.0: 256,
 150.0: 256,
 159.0: 256,
 174.0: 224,
 188.0: 256,
 196.0: 256,
 198.0: 256,
 204.0: 256,
 205.0: 256,
 208.0: 224,
 209.0: 256,
 224.0: 256,
 226.0: 224,
 240.0: 256,
 242.0: 256,
 248.0: 256,
 250.0: 256,
 254.0: 256,
 288.0: 256,
 296.0: 256,
 307.0: 224,
 403.0: 256,
 459.0: 256,
 472.0: 224,
 479.0: 224,
 480.0: 256,
 492.0: 256,
 510.0: 256,
 597.0: 256,
 636.0: 256,
 637.0: 256,
 652.0: 160,
 770.0: 256,
 868.0: 256,
 882.0: 256,
 887.0: 256,
 948.0: 256,
 949.0: 256,
 1050.0: 224,
 1052.0: 256,
 1066.0: 256,
 1112.0: 256,
 1116.0: 256,
 1121.0: 256,
 1160.0: 192,
 1182.0: 256,
 1212.0: 256,
 1235.0: 256,
 1246.0: 256,
 1259.0: 256,
 1271.0: 256,
 1272.0: 192,
 1335.0: 256,
 1383.0: 224,
 1392.0: 256,
 1417.0: 256,
 1425.0: 256,
 1472.0: 256,
 1536.0: 256,
 1607.0: 256,
 1737.0: 256,
 1841.0: 256,
 1851.0: 256,
 1867.0: 256,
 1874.0: 256,
 1926.0: 256,
 1961.0: 256,
 1963.0: 256,
 1970.0: 256,
 2012.0: 256,
 2060.0: 

## Modeling

### tpot

In [21]:
# Flatten X
X = X.reshape(X.shape[0], -1)
X.shape

(49664, 3672)

In [22]:
y.shape

(49664,)

In [23]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.75, test_size=0.25)
# Make a custom metric function
def my_custom_accuracy(y_true, y_pred):
    return float(sum(y_pred == y_true)) / len(y_true)

# Make a custom a scorer from the custom metric function
# Note: greater_is_better=False in make_scorer below would mean that the scoring function should be minimized.
# my_custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('./model/tpot/tpot_mfcc_pipeline_full.py')

                                                                                   
Generation 1 - Current best internal CV score: 0.7305629709316416
                                                                                   
Generation 2 - Current best internal CV score: 0.7305629709316416
                                                                                 
Generation 3 - Current best internal CV score: 0.7433691004873408
                                                                
Generation 4 - Current best internal CV score: 0.7433691004873408
                                                               
Generation 5 - Current best internal CV score: 0.8607980495557712
                                                              
Best pipeline: KNeighborsClassifier(MinMaxScaler(input_matrix), n_neighbors=86, p=2, weights=uniform)
0.8998872422680413
